In [1]:
from utils import *
from pca_analysis import *
from odds_import_df import *

In [2]:
import pandas as pd
from sklearn import tree
import os
from sklearn.preprocessing import StandardScaler

In [31]:
current_dir = os.getcwd()
folder = 'data/results_international'
split_df = current_dir+'/'+folder+'/split_df_teams.csv'
outcome = current_dir+'/'+folder+'/split_df_diff.csv'

In [45]:
data = load_data(split_df, draw=False )
outcome = data['Outcome'].values
data = pd.read_csv(split_df)
data['Outcome'] = outcome

In [53]:
data

,Date,Game ID,Team,Score,Possession - Overall,Possession - 1st half,Possession - 2nd half,Tries,Conversions,Penalties,...,Scrums Total,Penalties Conceded Own Half,Penalties Conceded Opp Half,Yellow Cards,Red Cards,% Rucks Successful,% Goal kicks Successful,% Scrums Successful,% Tackle Successful,Outcome
0,28-Oct-2023,87881,New Zealand,11,60,63,58,1,0,2,...,2,4,1,1,1,0.958333,0.500000,1.000000,0.867925,0.0
1,27-Oct-2023,87880,Argentina,23,53,45,59,2,2,3,...,7,5,2,0,0,0.981982,0.833333,0.857143,0.884298,1.0
2,21-Oct-2023,87879,England,15,56,52,59,0,0,4,...,7,4,7,0,0,0.985294,1.000000,0.428571,0.855769,0.0
3,20-Oct-2023,87878,Argentina,6,43,46,40,0,0,2,...,4,4,6,0,0,0.966387,1.000000,1.000000,0.790123,1.0
4,15-Oct-2023,87877,France,28,60,59,60,3,2,3,...,9,2,4,0,0,0.953704,0.714286,0.555556,0.884615,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,9-Sep-2023,87838,Argentina,10,50,46,54,1,1,1,...,2,5,8,1,0,0.927711,0.666667,0.500000,0.892473,0.0
92,9-Sep-2023,87837,Georgia,15,43,37,49,2,1,1,...,6,7,6,1,0,0.893617,0.666667,1.000000,0.895028,1.0
93,9-Sep-2023,87836,Romania,8,33,43,24,1,0,1,...,9,7,5,1,0,0.909091,0.500000,0.888889,0.803922,0.0
94,9-Sep-2023,87835,Namibia,8,48,53,44,1,0,1,...,6,5,8,1,0,0.962963,0.333333,1.000000,0.722973,1.0


In [80]:
#difference between winning and loosing
small = data.drop(columns=(['Game ID', 'Team', 'Date']))
wins= small[small['Outcome'] == 1]
loss= small[small['Outcome'] == 0]
loss_summary.columns = ['mean_loss', 'std_loss', '25%_loss', '75%_loss']
win_summary.columns = ['mean_win', 'std_win', '25%_win', '75%_win']
loss_summary = loss_summary.round(2)
win_summary = win_summary.round(2)

# Concatenate the DataFrames side by side
summary = pd.concat([loss_summary, win_summary], axis=1)

# Calculate mean ± std for loss and win
summary['mean_loss ± std'] = summary['mean_loss'].astype(str) + ' ± ' + summary['std_loss'].astype(str)
summary['mean_win ± std'] = summary['mean_win'].astype(str) + ' ± ' + summary['std_win'].astype(str)

# Select and reorder the columns to include the mean±std columns
final_summary = summary[['mean_loss ± std', 'mean_win ± std']]

# Write the result to an Excel file
final_summary.to_excel('summary.xlsx', index=True)

# Display the result (optional)
print(final_summary)
final_summary.to_excel('data/results_international'+'/summary.xlsx')

                            mean_loss ± std   mean_win ± std
Score                          25.6 ± 18.54    28.91 ± 24.55
Possession - Overall           50.94 ± 7.29      49.3 ± 7.49
Possession - 1st half          50.09 ± 9.13      50.0 ± 9.91
Possession - 2nd half         51.79 ± 10.14    48.62 ± 10.01
Tries                           3.09 ± 2.92       3.7 ± 3.84
Conversions                     2.43 ± 2.58       2.89 ± 3.4
Penalties                       1.64 ± 1.67      1.45 ± 1.49
Metres                       452.23 ± 178.1  491.62 ± 201.17
Defenders Beaten              23.04 ± 10.91    24.91 ± 11.54
Clean breaks                    5.72 ± 4.22       7.38 ± 5.4
Gain line Carries              58.21 ± 20.2    60.02 ± 20.63
Passes                       142.68 ± 55.14    140.6 ± 41.33
Offloads                        6.51 ± 4.55      7.36 ± 4.87
Turnovers won                   5.85 ± 2.41      5.81 ± 2.53
Kicks from Hand                 24.85 ± 8.6     24.64 ± 8.22
Rucks Won               

In [200]:
data = pd.read_csv(split_df )
variables_to_remove = ['Game ID', 'Date', 'Team', 'Score','Tries', 'Conversions','Goal Kicks Successful', 'Goal Kicks Attempted']
data_small = data.drop(columns=variables_to_remove)

In [210]:
scaler = StandardScaler()
features = data_small.columns
data_scaled = scaler.fit_transform(data_small)
data_scaled = pd.DataFrame(data=data_scaled, columns=features)

In [243]:
corr = data_scaled.corr()[['Passes', 'Metres', 'Kicks from Hand']]
corr = corr[corr < 1]
corr =corr[(corr < 1) & ((corr > 0.6) | (corr <-0.6))].unstack().transpose()\
    .sort_values( ascending=False)\
    .drop_duplicates()

print(corr)

Metres  Gain line Carries        0.867695
Passes  Gain line Carries        0.850585
Metres  Clean breaks             0.813799
        Offloads                 0.755190
Passes  Metres                   0.743357
Metres  Defenders Beaten         0.733678
Passes  Rucks Won                0.695583
        Total Rucks              0.695325
        Defenders Beaten         0.693512
        Possession - Overall     0.657686
        Offloads                 0.654279
        Clean breaks             0.638834
Metres  Possession - Overall     0.630042
Passes  Possession - 1st half         NaN
dtype: float64


In [224]:

print(corr)

                               Passes    Metres  Kicks from Hand
Possession - Overall         0.657686  0.630042         0.076002
Possession - 1st half        0.479469  0.518302         0.136369
Possession - 2nd half        0.509314  0.439098        -0.009382
Penalties                   -0.300732 -0.310202         0.458108
Metres                       0.743357  1.000000         0.023179
Defenders Beaten             0.693512  0.733678        -0.106224
Clean breaks                 0.638834  0.813799        -0.028888
Gain line Carries            0.850585  0.867695        -0.089076
Passes                       1.000000  0.743357        -0.135372
Offloads                     0.654279  0.755190        -0.068742
Turnovers won               -0.055877  0.067765         0.034082
Kicks from Hand             -0.135372  0.023179         1.000000
Rucks Won                    0.695583  0.396525        -0.043914
Tackles Made                -0.212865 -0.236454         0.092552
Tackles Missed           